In [ ]:
!jt -t grade3 -cellw 90% -ofs 12

### Import libs

In [ ]:
import matplotlib
import numpy as np
%matplotlib widget
from matplotlib.pyplot import figure, draw
import matplotlib.pyplot as plt
import math
import matplotlib.animation as animation
from IPython.display import HTML

LVC_path = '/misc/home/u1220/analysis_jupyter'
import sys
sys.path.remove('/home/u1220/lvd_aniso/jupiter_books') if '/home/u1220/lvd_aniso/jupiter_books' in sys.path else None
sys.path.append(LVC_path) if not LVC_path in sys.path else None

from LVC.LVCcase_cabel import CabelCase
plt.style.use('LVC')

### Load data

In [ ]:
proj = CabelCase(project_folder=r'/misc/home/u1220/lvd_aniso/1D_APmu/results/id_2225/APmu_orig0_')
proj.read_input()

In [ ]:
proj.load_udat(verbose=True)

### Heatmap

In [ ]:
fig = figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')

proj.draw_heatmap(plt, vmax=25)
# plt.xlim([30000, 31000])

### Animation

In [ ]:
def draw_frame(i):
    plt.yticks([])
    plt.xticks(range(0, int(proj.N*proj.dr) + 1, 25 ))
    return plt.imshow(get_frame(i), origin='lower', interpolation='none', extent=[0, proj.N*proj.dr, 0, 1], animated=True, 
                      vmin=np.amin(proj.udat), vmax=100)


def get_frame(i):
    return np.array([proj.udat[i,:]])

fig = figure(num=None, figsize=(8, 2), dpi=80, facecolor='w')
draw_frame(10)

In [ ]:
def animate(i):
    canvas.set_data(get_frame(i))
    return canvas,

fig = figure(num=None, figsize=(8, 2), dpi=80, facecolor='w')
canvas = draw_frame(0)

%matplotlib
ani = animation.FuncAnimation(fig, animate, frames=range(0, 2000, 2), repeat=False)
%matplotlib notebook
# ani.save("/misc/home1/u1220/lvc/analysis_jupyter/_trash/1D_Wenckebach.gif",  writer='imagemagick', fps=60)

HTML(ani.to_jshtml())

### Potential

In [ ]:
# point_x = 3
# x_ind = int(point_x/(proj.mult*proj.dr))
x_ind = int(proj.udat[0].size/2)

fig = figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')
plt.grid()
plt.title("Potential in time at point %.2f" % (x_ind * proj.dr * proj.mult), fontsize=14)
plt.xlabel('Time, ms', fontsize=14)
plt.ylabel('Potential, mV', fontsize=14)
plt.tight_layout()
# plt.xlim([50000, 51000])
plt.plot(np.arange(0, proj.time, proj.dT), proj.udat[:,x_ind])

### APD

In [ ]:
# get APDX (by percentage)

x, start_t, APD_percent = (125, 100, 0.9)

x_ind = int(x/(proj.mult*proj.dr))
start_t_ind = int(start_t/proj.dT)

times, apds, dis = proj.calculate_APD_by_percentage_with_DI(x=x_ind, start=start_t_ind, APD_percent=.9)
print(apds)
# print(dis)

In [ ]:
# get APD_X
x, start_t, APD_level = (125, 100, -70)

x_ind = int(x/(proj.mult*proj.dr))
start_t_ind = int(start_t/proj.dT)

times, apds = proj.calculate_APD_X(x=x_ind, start=start_t_ind, u_level=APD_level)
print(apds)

### Periods

In [ ]:
x, start_t, u_level = (8, 0, .2)
start_t_ind = int(start_t/proj.dT)
times, periods = proj.calculate_periods(x=x, u_level=u_level, start=start_t_ind)
fig = figure(num=None, figsize=(14, 5), dpi=80, facecolor='w')
plt.scatter(times, periods, s=5)
plt.grid()

### Velocity

In [ ]:
# get velocity

vel = proj.get_velocity(left=30, right=220, u_level=20, verbose=True)
print(round(vel,2))

### Measured periods

In [ ]:
c = 40
(x1, x2, y1, y2) = (200, 360, 200, 500)
figure(num=None, figsize=((x2-x1)/c+1, (y2-y1)/c), dpi=80, facecolor='w')
# figure(num=None, figsize=(8, 8), dpi=80, facecolor='w')

res = proj.get_measured_vs_stimulated(_error=.4)

plt.scatter(res[:,0], res[:,2], marker='o', color='red', s=3)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Measured period at point 50 mm", fontsize=16)
plt.xlabel("Stimulation period, ms", fontsize=16)
plt.ylabel("Measured period, ms", fontsize=16)
plt.grid()
# plt.yticks(range(y1,y2+1,40))
# plt.xticks(range(x1,x2+1,40))
plt.xlim(x1, x2)
plt.ylim(y1, y2)
plt.tight_layout()

plt.show()